In [ ]:
from NovelSwarmBehavior.novel_swarms.behavior.AngularMomentum import AngularMomentumBehavior
from NovelSwarmBehavior.novel_swarms.behavior.AverageSpeed import AverageSpeedBehavior
from NovelSwarmBehavior.novel_swarms.behavior.GroupRotationBehavior import GroupRotationBehavior
from NovelSwarmBehavior.novel_swarms.behavior.RadialVariance import RadialVarianceBehavior
from NovelSwarmBehavior.novel_swarms.behavior.ScatterBehavior import ScatterBehavior
from NovelSwarmBehavior.novel_swarms.behavior.SensorOffset import GeneElementDifference
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from generation.evolution import ModifiedHaltingEvolution, ModifiedNoveltyArchieve
from generation.halted_evolution import HaltedEvolution
from data.swarmset import SwarmDataset, DataBuilder
import numpy as np
from scipy import ndimage
import cv2
import pygame

def resizeInput(X, w=200):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(w, w), interpolation=cv2.INTER_AREA)
    return resized

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

In [5]:
import numpy as np
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
import time
from networks.ensemble import Ensemble

trial_name = f"{str(int(time.time()))}"

TRAIN = True
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ensemble = Ensemble(size=3, output_size=5, lr=30e-4, learning_decay=0.9, decay_step=4, threshold=9.0, weight_decay=1e-6, new_model=True)
ensemble.load_ensemble("final-original-72")
# ensemble.load_ensemble("full-mini-HIL")
network = ensemble.ensemble[0]
network.eval()

SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = False
EVOLUTION = True
EPOCHS = 20
DATA_SIZE = 10000
EVOLUTIONS = 50
K_CLUSTERS = 8

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full-mini-dual", rank=0)
evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=1200, n_agents=24, evolve_population=100, seed=None)
dataset = anchor_dataset if EVOLUTION else sampled_dataset
random_archive = DataAggregationArchive(scalar=True)

hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()

for gen in range(EVOLUTIONS):
    # Simulate current population + Save Data
    for i in range(len(evolution.getPopulation())):
        # The collection of the original behavior vector below is only used to collect data to compare with the baseline
        visual_behavior, genome, baseline_behavior = evolution.next()
        dataset.new_entry(visual_behavior, genome, baseline_behavior)

    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

    # Then, evolve
    start_time = time.time()
    embedded_archive = hil.getEmbeddedArchive(dataset, network)
    evolution.overwriteArchive(embedded_archive, random_archive)
    embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
    evolution.overwriteBehavior(embedded_behavior)
    evolution.saveArchive(trial_name)
    evolution.evolve()
    evolution.restart_screen()

    print(f"Evolution complete for gen{gen}")

    # Cluster current dataset, display clusters, and save for analysis
    if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
        hil.embed_and_cluster(network, dataset, auto_quit=True)
        evolution.restart_screen()

HIL Init!
<class 'generation.evolution.ModifiedNoveltyArchieve'>
[ 18.25682631  12.0771122   19.87911616  10.82181965  14.11057124
  11.20057317  15.83962609  12.83254781  44.35411206  20.9249553
  28.63162451  47.94084988  18.10967529  11.81132867  12.6460169
  12.77727091  18.45890565  20.03955916 192.55466101  33.21808338
  58.78641454  50.16675147  13.45705162  42.15791916  22.84834167
  16.54335143  12.24213026  10.72612677  22.50689394  45.63781188
  11.11024235  12.57293137  42.55125332  16.71797382  25.29736231
  11.44273065 103.83979384  17.66531     67.02628922  28.59282725
  14.98298217 163.99162396  26.07371349  59.69421106  15.71472405
  12.86646101  12.22119968  11.36325039  12.22122842  16.44892853
  13.19659892  10.12697766  10.52764647  30.36816517  12.84324427
  20.36714296  20.54440743  19.75847012  27.43263643  20.44725488
  19.25862883  15.04349607  35.47889199  19.37492989  15.97815688
  27.06436621  14.11067302  13.05961172  19.29591629  12.0783122
  12.31453345 

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[ 11.33965486  21.98058755  14.38018369   9.94207663 171.65745595
   8.06852613  18.96750699  17.17889507  14.3970109   17.53513666
  23.08094838   9.61445194   6.66452202  22.97561791  25.91610947
  11.26513175   8.77101366  18.98554826  29.98995398  59.9214194
   6.80506133  20.08023555  17.23659909   8.76030051   8.6150926
  24.1687629   23.72970323  12.9091728    9.27358461   6.54820898
  60.11574089  15.0124553   20.99134547   7.03415422  35.8340483
  11.587706    18.94928867  10.45538186  21.65597436   9.19623908
  96.98730525  17.35297194  25.94588937  14.835651     6.93384479
  89.61819161  19.0649228    7.89880024  26.48931422  17.86925067
  16.02070112  29.69022525   7.82999262   9.29802822  29.12785721
  18.19545523  33.9339133    7.59871502  13.31177914   8.92300252
  17.06786075  34.17593928  10.54904934  11.20303711  18.90868134
  12.31247755  44.7423895   16.68000546   8.43960186  23.34056664
  27.73864874  49.801556

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[ 16.64171621   6.91639047  11.76399191  22.40225383  20.08112775
  17.28524501  19.26313967  16.59594986  17.38708351  10.99395694
  34.20293881  11.07186563 120.17902508  10.23469343  10.6589443
  25.71489408  13.63097807  13.09122925  14.0378699   47.23855984
  13.35062009   9.87363397   9.97821091  24.51155343   8.8827977
  11.13837575  20.10308539  13.65333679 239.58785289  18.90126013
  12.70513801  18.16363119  16.8991226   43.39221184  13.61469686
  19.28804418  19.09401317   7.49082967  39.79522019  15.71071657
  20.11715273   7.91894006  12.37918585  14.17534823  20.20537057
  17.51579675  18.32342942  26.02884671  18.08622325  34.32396186
  25.18150232  12.39238556  17.09418126  11.67304689  12.19528988
  13.0210016   37.82725254  30.19621101  12.75686816  34.37501153
  17.62241254  12.3596904   14.65663514  14.13202266  31.64345865
  14.87322966   8.01492564   6.24521953  27.23877206  12.43429002
  38.20915504 238.96074

...

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[ 13.05052327   8.79732136  13.0060411   15.59855579  12.69642383
  15.84489637  11.81266031  15.49854392  28.31945433  15.98508096
  12.80605121  11.68580894 147.50678158  16.58625254  26.91227086
  13.93267521  16.0083369   19.74669444  19.79642706  18.57422074
  10.14449648  35.58510909  11.33892138  47.16811271  18.00765882
  15.76406953  21.25362604  13.72057057  14.79828092  19.46378218
   5.71379402  22.6799052   17.53197055  43.81053589  17.18168616
  10.92283977  13.6093502   16.64882551  20.9562206   21.38091283
  22.63784273  19.28751927  11.84996408  22.25843398  14.98445166
  15.75197457   8.6754818   16.19490898  12.39244732  26.05920504
  10.13039831  61.10559996  15.4032951   16.37782554  16.02722755
  23.73582756   2.81474526  15.86718317   5.89664656  19.43873989
   8.99033181  41.99992311   8.91726312  24.58672902  17.39575037
  14.14287386  33.26578721  33.71371716  41.05339404  14.73223386
  21.08171632  32.252

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[15.06520524 10.71653991 18.94159845 14.27242877 50.60090086 10.28448795
 13.44389425 21.19816555 11.91509488 13.07916299 21.84245351  8.89482131
 13.33490508 11.64042613 16.49213317 44.12182808 50.03339681 14.38298234
  8.22356323 11.49484573  7.48953733 17.65940456 13.64573997 31.61057498
 12.00949255 17.47098514 14.7278351  18.70182201 25.25534495 11.24382109
  5.15646365 60.72149474 66.94942803 13.88171089 27.46920982 17.1258066
 12.25867107 16.71090876 53.45673039 17.00712848 12.6877975  16.96676397
 42.15673101 28.90246076 11.49970296 30.39205841 21.90394961 19.03651557
 31.77163974 56.23337165 12.68268174  7.85579181 24.7607523  86.20880581
 67.76745825 26.43650717 15.3520715  48.3769011  21.29002611 24.85441361
 34.15737412 35.75633725  9.85003102 15.83252461 37.45737102 12.80349991
 16.04522617 22.93968567 22.71668493 15.90353514 33.4852318  21.17676269
 10.52545248 11.03130028 16.19408053 26.67492943 15.0295979  46.013943

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [ ]:
from ui.clustering_gui import ClusteringGUI
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()